<a href="https://colab.research.google.com/github/JKaly-prog/ML-Emotor/blob/first_branch/tdms_mat_to_parquet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Code to reduce the size of emotors measurement samples**

In [2]:
pip install nptdms

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nptdms: filename=nptdms-1.10.0-py3-none-any.whl size=108456 sha256=1ac0be2819a85dc5ff86a194bc09bb6c6cf1a0575ca75ee448179066722a140a
  Stored in directory: /root/.cache/pip/wheels/a8/06/c1/f72c8e132b2d7b2bae5bff097aa2b80086fa9873c2b1bbc98c
Successfully built nptdms


### Visualizing .tdms and .mat files from Google Drive files

In [3]:
import numpy as np
import pandas as pd
from nptdms import TdmsFile
from google.cloud import storage

In [4]:
# df = pd.read_csv('/content/drive/MyDrive/Emotor_short/0Nm_BPFI_03.tdms') # This caused a UnicodeDecodeError

with TdmsFile.open('/content/drive/MyDrive/Emotor_short/0Nm_BPFI_03.tdms') as tdms_file:
    # You can inspect the TDMS file structure:
    # print(tdms_file)

    # To get a DataFrame from a specific group or channel, you need to specify it.
    # For example, to get all data as a DataFrame:
    # (Note: This might need adjustment based on your specific TDMS file structure)
    df_03 = tdms_file.as_dataframe()

    # Display the first few rows of the DataFrame
    display(df_03.head())

    # You might want to access specific groups or channels like this:
    # group = tdms_file['Measured Data']
    # channel = group['Voltage']
    # channel_data = channel.data
    # df = pd.DataFrame({'Voltage': channel_data})

    print(df_03.shape)

,/'Log'/'cDAQ9185-1F486B5Mod1/ai0',/'Log'/'cDAQ9185-1F486B5Mod1/ai1',/'Log'/'cDAQ9185-1F486B5Mod2/ai0',/'Log'/'cDAQ9185-1F486B5Mod2/ai2',/'Log'/'cDAQ9185-1F486B5Mod2/ai3'
0,27.607992,28.217591,1.894377,0.949463,-2.165271
1,27.607992,28.217591,2.128889,0.926111,-2.309938
2,27.607992,28.217591,2.373001,0.882154,-2.526938
3,27.607992,28.217591,2.087747,1.193974,-2.444867
4,27.607992,28.217591,2.393572,1.125291,-2.718899


(1536492, 5)


In [5]:
from nptdms import TdmsFile

In [6]:
tdms = TdmsFile.read("/content/drive/MyDrive/Emotor_short/0Nm_BPFI_03.tdms")

print(tdms.groups())

[<TdmsGroup with path /'Test Information'>, <TdmsGroup with path /'Log'>]


In [10]:
group = tdms["Log"]

for channel in group.channels():
    print(channel.name)

cDAQ9185-1F486B5Mod1/ai0
cDAQ9185-1F486B5Mod1/ai1
cDAQ9185-1F486B5Mod2/ai0
cDAQ9185-1F486B5Mod2/ai2
cDAQ9185-1F486B5Mod2/ai3


In [11]:
for channel in group.channels():
    print("Canal:", channel.name)
    print("   Props:", channel.properties)
    print()

Canal: cDAQ9185-1F486B5Mod1/ai0
   Props: OrderedDict({'wf_xname': 'Time', 'wf_xunit_string': 's', 'wf_samples': 25608, 'unit_string': '°C', 'NI_DeviceName': 'cDAQ9185-1F486B5Mod1', 'DAC~Channel~Id': 'ai0', 'DAC~Channel~Type': 'Temperature', 'DAC~Channel~Sensor': 'Thermocouple', 'DAC~Device~Name': 'NI 9210 (miniTC)', 'DAC~Device~Id': '01F4C308', 'DAC~Chassis~Name': 'cDAQ9185-1F486B5', 'DAC~Chassis~Id': '01F486B5', 'DAC~Channel~HighRange': 1372.0, 'DAC~Channel~LowRange': -200.0, 'NI_SensorScale~ScaleType': 'Linear', 'NI_SensorScale~SensorSlope': 1.0, 'NI_SensorScale~Y-Intercept': 0.0, 'NI_ThermocoupleType': 'K', 'NI_TerminalConfiguration': 'Differential', 'NI_CjcSource': 'BuiltIn', 'NI_CjcTemperature': 25.0, 'wf_increment': 3.905e-05, 'wf_start_offset': 0.0, 'wf_start_time': np.datetime64('2021-06-28T11:02:43.489671'), 'minimum': 27.597912421141356, 'maximum': 28.527226435741586, 'NI_ChannelValueAverage': 28.077345520807725, 'monotony': 'not monotone', 'novaluekey': 'No'})

Canal: cDAQ9

In [13]:
import numpy as np
from scipy.io import loadmat
import pandas as pd

# Load file and access the main structured array for the signal
data = loadmat('/content/drive/MyDrive/Emotor_short/0Nm_BPFI_10.mat')
signal = data["Signal"][0][0]

# ===== Extract X axes parameters =====
# These values are nested within the 'x_values' structured array
start = float(signal["x_values"][0][0]["start_value"][0][0])
dt    = float(signal["x_values"][0][0]["increment"][0][0])
n     = int(signal["x_values"][0][0]["number_of_values"][0][0])

# Generate the time array
time = start + np.arange(n) * dt

# ===== Extract Y axes (signal) values =====
# The actual signal data is nested within 'y_values'
vals = signal["y_values"][0][0]["values"]

# Handle potential multi-nesting (e.g., if 'values' is an array of object arrays)
while isinstance(vals, np.ndarray) and vals.dtype == object:
    vals = vals.item()

# Convert to a NumPy 2D array if it's not already
values = np.array(vals)

# ===== Build DataFrame =====
# Create DataFrame from signal values, and then add the time column
df = pd.DataFrame(values, columns=[f"ch{i+1}" for i in range(values.shape[1])])
df["time"] = time

display(df.head())
print(df.shape)

,ch1,ch2,ch3,ch4,time
0,-13.951874,11.725489,8.098866,-1.288651,0.000013
1,-4.841047,0.135855,6.042726,3.253262,0.000052
2,8.094253,2.141985,6.401257,1.619741,0.000091
3,10.597587,3.338727,6.948811,-1.191469,0.000130
4,16.695230,-5.705184,1.261864,1.408720,0.000169


(1536000, 5)


In [14]:
with TdmsFile.open('/content/drive/MyDrive/Emotor_short/0Nm_BPFI_10.tdms') as tdms_file:
  df_10 = tdms_file.as_dataframe()
  display(df_10.head())
  print(df_10.shape)

,/'Log'/'cDAQ9185-1F486B5Mod1/ai0',/'Log'/'cDAQ9185-1F486B5Mod1/ai1',/'Log'/'cDAQ9185-1F486B5Mod2/ai0',/'Log'/'cDAQ9185-1F486B5Mod2/ai2',/'Log'/'cDAQ9185-1F486B5Mod2/ai3'
0,26.914319,28.366138,-1.019880,-2.378906,2.635163
1,26.914319,28.366138,-0.856681,-2.458578,2.582304
2,26.914319,28.366138,-0.738739,-2.521766,2.505798
3,26.914319,28.366138,-0.980109,-2.425610,2.656029
4,26.914319,28.366138,-1.225592,-2.365169,2.789567


(1536492, 5)


In [15]:
with TdmsFile.open('/content/drive/MyDrive/Emotor_short/0Nm_BPFI_30.tdms') as tdms_file:
  df_30 = tdms_file.as_dataframe()
  display(df_30.head())
  print(df_30.shape)

,/'Log'/'cDAQ9185-1F486B5Mod1/ai0',/'Log'/'cDAQ9185-1F486B5Mod1/ai1',/'Log'/'cDAQ9185-1F486B5Mod2/ai0',/'Log'/'cDAQ9185-1F486B5Mod2/ai2',/'Log'/'cDAQ9185-1F486B5Mod2/ai3'
0,26.947089,26.958406,2.403172,0.839570,-2.330803
1,26.947089,26.958406,2.091861,0.886275,-2.284899
2,26.947089,26.958406,2.127518,0.651380,-2.117976
3,26.947089,26.958406,2.286602,0.544234,-2.172226
4,26.947089,26.958406,2.551285,0.538740,-2.371143


(1536492, 5)


In [16]:
from scipy.io import loadmat
import pandas as pd
import h5py

In [17]:
loadmat('/content/drive/MyDrive/Emotor_short/0Nm_BPFI_10.mat')

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN, Created on: Mon Jun 28 22:02:16 2021',
 '__version__': '1.0',
 '__globals__': [],
 'Signal': array([[(array([[(array([[1.29076899e-05]]), array([[3.90625e-05]]), array([[1536000]], dtype=int32), array([[(array(['s'], dtype='<U1'), array([[(array([[0]], dtype=uint8), array([[1]], dtype=uint8), array([[1]], dtype=uint8))]],
                               dtype=[('offset', 'O'), ('factor', 'O'), ('log_reference', 'O')]), array([[(array([[0]], dtype=uint8), array([[1]], dtype=uint8), array(['LENGTH'], dtype='<U6')),
                                 (array([[0]], dtype=uint8), array([[1]], dtype=uint8), array(['ANGLE'], dtype='<U5')),
                                 (array([[0]], dtype=uint8), array([[1]], dtype=uint8), array(['MASS'], dtype='<U4')),
                                 (array([[1]], dtype=uint8), array([[1]], dtype=uint8), array(['TIME'], dtype='<U4')),
                                 (array([[0]], dtype=uint8), array([[

In [18]:
signal = data["Signal"][0][0]

print("x_values:\n", signal["x_values"])
print("\nstart_value:\n", signal["x_values"][0][0]["start_value"])
print("\nincrement:\n", signal["x_values"][0][0]["increment"])
print("\nnumber_of_values:\n", signal["x_values"][0][0]["number_of_values"])


x_values:
 [[(array([[1.29076899e-05]]), array([[3.90625e-05]]), array([[1536000]], dtype=int32), array([[(array(['s'], dtype='<U1'), array([[(array([[0]], dtype=uint8), array([[1]], dtype=uint8), array([[1]], dtype=uint8))]],
                dtype=[('offset', 'O'), ('factor', 'O'), ('log_reference', 'O')]), array([[(array([[0]], dtype=uint8), array([[1]], dtype=uint8), array(['LENGTH'], dtype='<U6')),
                  (array([[0]], dtype=uint8), array([[1]], dtype=uint8), array(['ANGLE'], dtype='<U5')),
                  (array([[0]], dtype=uint8), array([[1]], dtype=uint8), array(['MASS'], dtype='<U4')),
                  (array([[1]], dtype=uint8), array([[1]], dtype=uint8), array(['TIME'], dtype='<U4')),
                  (array([[0]], dtype=uint8), array([[1]], dtype=uint8), array(['CURRENT'], dtype='<U7')),
                  (array([[0]], dtype=uint8), array([[1]], dtype=uint8), array(['LIGHT'], dtype='<U5')),
                  (array([[0]], dtype=uint8), array([[1]], dtype=uint

# Connection with nucked in GCP to extract .tdms and .mat files

In [21]:
from google.colab import auth
auth.authenticate_user()

In [23]:
!pip install google-cloud-storage

In [24]:
from google.cloud import storage

client = storage.Client()
bucket = client.get_bucket("emotor-dataset-raw")  # tu bucket

for blob in bucket.list_blobs():
    print(blob.name)

current,temp/0Nm_BPFI_03.tdms
current,temp/0Nm_BPFI_10.tdms
current,temp/0Nm_BPFI_30.tdms
current,temp/0Nm_BPFO_03.tdms
current,temp/0Nm_BPFO_10.tdms
current,temp/0Nm_BPFO_30.tdms
current,temp/0Nm_Misalign_01.tdms
current,temp/0Nm_Misalign_03.tdms
current,temp/0Nm_Misalign_05.tdms
current,temp/0Nm_Normal.tdms
current,temp/0Nm_Unbalance_0583mg.tdms
current,temp/0Nm_Unbalance_1169mg.tdms
current,temp/0Nm_Unbalance_1751mg.tdms
current,temp/0Nm_Unbalance_2239mg.tdms
current,temp/0Nm_Unbalance_3318mg.tdms
current,temp/2Nm_BPFI_03.tdms
current,temp/2Nm_BPFI_10.tdms
current,temp/2Nm_BPFI_30.tdms
current,temp/2Nm_BPFO_03.tdms
current,temp/2Nm_BPFO_10.tdms
current,temp/2Nm_BPFO_30.tdms
current,temp/2Nm_Misalign_01.tdms
current,temp/2Nm_Misalign_03.tdms
current,temp/2Nm_Misalign_05.tdms
current,temp/2Nm_Normal.tdms
current,temp/2Nm_Unbalance_0583mg.tdms
current,temp/2Nm_Unbalance_1169mg.tdms
current,temp/2Nm_Unbalance_1751mg.tdms
current,temp/2Nm_Unbalance_2239mg.tdms
current,temp/2Nm_Unbalance_

In [26]:
blob = bucket.blob("current,temp/0Nm_BPFI_03.tdms")
tdms_bytes = blob.download_as_bytes()

from nptdms import TdmsFile
import io

tdms = TdmsFile(io.BytesIO(tdms_bytes))

for group in tdms.groups():
    print("Group:", group.name)
    for channel in group.channels():
        print("  - Channel:", channel.name)

Group: Test Information
Group: Log
  - Channel: cDAQ9185-1F486B5Mod1/ai0
  - Channel: cDAQ9185-1F486B5Mod1/ai1
  - Channel: cDAQ9185-1F486B5Mod2/ai0
  - Channel: cDAQ9185-1F486B5Mod2/ai2
  - Channel: cDAQ9185-1F486B5Mod2/ai3


In [27]:
# ============================================
# 1. AUTENTICACIÓN
# ============================================
from google.colab import auth
auth.authenticate_user()

# ============================================
# 2. IMPORTS
# ============================================
import io
import pandas as pd
from nptdms import TdmsFile
from google.cloud import storage

BUCKET_NAME = "emotor-dataset-raw"
INPUT_PREFIX = "current,temp/"
OUTPUT_PREFIX = "current_temp_short/"

client = storage.Client()
bucket = client.bucket(BUCKET_NAME)

# ============================================
# 3. LISTAR ARCHIVOS TDMS
# ============================================
def list_tdms_files():
    return [
        blob.name
        for blob in bucket.list_blobs(prefix=INPUT_PREFIX)
        if blob.name.endswith(".tdms")
    ]

tdms_files = list_tdms_files()
print(f"Se encontraron {len(tdms_files)} archivos TDMS.")


Se encontraron 45 archivos TDMS.


In [28]:
def convert_tdms_to_parquet(blob_name):
    print(f"\n📄 Procesando: {blob_name}")

    # Descargar archivo TDMS desde GCS
    blob = bucket.blob(blob_name)
    tdms_bytes = blob.download_as_bytes()

    # Leer TDMS
    tdms = TdmsFile(io.BytesIO(tdms_bytes))

    # Seleccionar el primer grupo (casi siempre solo hay uno)
    group = tdms.groups()[0]
    df = group.as_dataframe()

    # Crear nombre parquet
    parquet_name = blob_name.replace(INPUT_PREFIX, OUTPUT_PREFIX).replace(".tdms", ".parquet")

    # Convertir a parquet local
    df.to_parquet("temp.parquet")

    # Subir el parquet al bucket
    out_blob = bucket.blob(parquet_name)
    out_blob.upload_from_filename("temp.parquet")

    print(f"✅ Convertido y subido: {parquet_name}")


In [29]:
for blob_name in tdms_files:
    convert_tdms_to_parquet(blob_name)

print("\n🎉 ¡Proceso completado! Todos los TDMS fueron convertidos a Parquet.")


📄 Procesando: current,temp/0Nm_BPFI_03.tdms
✅ Convertido y subido: current_temp_short/0Nm_BPFI_03.parquet

📄 Procesando: current,temp/0Nm_BPFI_10.tdms
✅ Convertido y subido: current_temp_short/0Nm_BPFI_10.parquet

📄 Procesando: current,temp/0Nm_BPFI_30.tdms
✅ Convertido y subido: current_temp_short/0Nm_BPFI_30.parquet

📄 Procesando: current,temp/0Nm_BPFO_03.tdms
✅ Convertido y subido: current_temp_short/0Nm_BPFO_03.parquet

📄 Procesando: current,temp/0Nm_BPFO_10.tdms
✅ Convertido y subido: current_temp_short/0Nm_BPFO_10.parquet

📄 Procesando: current,temp/0Nm_BPFO_30.tdms
✅ Convertido y subido: current_temp_short/0Nm_BPFO_30.parquet

📄 Procesando: current,temp/0Nm_Misalign_01.tdms
✅ Convertido y subido: current_temp_short/0Nm_Misalign_01.parquet

📄 Procesando: current,temp/0Nm_Misalign_03.tdms
✅ Convertido y subido: current_temp_short/0Nm_Misalign_03.parquet

📄 Procesando: current,temp/0Nm_Misalign_05.tdms
✅ Convertido y subido: current_temp_short/0Nm_Misalign_05.parquet

📄 Procesando

### **Convert .mat to .parquet**

In [31]:
# Libraries authentication

from google.colab import auth
auth.authenticate_user()

!pip install scipy google-cloud-storage

In [32]:
# Import + set

import io
import pandas as pd
from scipy.io import loadmat
from google.cloud import storage

BUCKET_NAME = "emotor-dataset-raw"
INPUT_PREFIX = "vibration/"
OUTPUT_PREFIX = "vibration_temp_short/"

client = storage.Client()
bucket = client.bucket(BUCKET_NAME)

In [33]:
# List .mat files within vibration

def list_mat_files():
    return [
        blob.name
        for blob in bucket.list_blobs(prefix=INPUT_PREFIX)
        if blob.name.endswith(".mat")
    ]

mat_files = list_mat_files()
print(f"Se encontraron {len(mat_files)} archivos MAT.")

Se encontraron 45 archivos MAT.


In [38]:
# .mat → dataframe → .parquet

def convert_mat_to_parquet(blob_name):
    print(f"\n─── Procesando: {blob_name}")

    # Descargar archivo MAT desde GCS
    blob = bucket.blob(blob_name)
    mat_bytes = blob.download_as_bytes()

    # Leer MAT
    mat = loadmat(io.BytesIO(mat_bytes))

    # Explicitly access the 'Signal' structure as seen in the exploratory cell
    # Assuming 'Signal' is the main key containing the relevant data
    if 'Signal' not in mat:
        print(f"⚠️ No se encontró la clave 'Signal' en el archivo {blob_name}.")
        return

    # Extract the nested 'Signal' structured array
    signal = mat["Signal"][0][0]

    # ===== Extract X axes parameters =====
    # These values are nested within the 'x_values' structured array
    start = float(signal["x_values"][0][0]["start_value"][0][0])
    dt     = float(signal["x_values"][0][0]["increment"][0][0])
    n      = int(signal["x_values"][0][0]["number_of_values"][0][0])

    # Generate the time array
    time = start + np.arange(n) * dt

    # ===== Extract Y axes (signal) values =====
    # The actual signal data is nested within 'y_values'
    vals = signal["y_values"][0][0]["values"]

    # Handle potential multi-nesting (e.g., if 'values' is an array of object arrays)
    while isinstance(vals, np.ndarray) and vals.dtype == object:
        vals = vals.item()

    # Convert to a NumPy 2D array if it's not already
    values = np.array(vals)

    # ===== Build DataFrame =====
    # Create DataFrame from signal values, and then add the time column
    df = pd.DataFrame(values, columns=[f"ch{i+1}" for i in range(values.shape[1])])
    df["time"] = time

    # Crear nombre parquet
    parquet_name = blob_name.replace(INPUT_PREFIX, OUTPUT_PREFIX).replace(".mat", ".parquet")

    # Guardar a parquet local
    df.to_parquet("temp.parquet")

    # Subir parquet al bucket
    out_blob = bucket.blob(parquet_name)
    out_blob.upload_from_filename("temp.parquet")

    print(f"✅ Convertido y subido a: {parquet_name}")

In [39]:
# Run all pipeline
for blob_name in mat_files:
    convert_mat_to_parquet(blob_name)

print("\n❌ ¡Conversión completa! Todos los MAT fueron convertidos a Parquet y subidos a vibration_temp_short/")


─── Procesando: vibration/0Nm_BPFI_03.mat
✅ Convertido y subido a: vibration_temp_short/0Nm_BPFI_03.parquet

─── Procesando: vibration/0Nm_BPFI_10.mat
✅ Convertido y subido a: vibration_temp_short/0Nm_BPFI_10.parquet

─── Procesando: vibration/0Nm_BPFI_30.mat
✅ Convertido y subido a: vibration_temp_short/0Nm_BPFI_30.parquet

─── Procesando: vibration/0Nm_BPFO_03.mat
✅ Convertido y subido a: vibration_temp_short/0Nm_BPFO_03.parquet

─── Procesando: vibration/0Nm_BPFO_10.mat
✅ Convertido y subido a: vibration_temp_short/0Nm_BPFO_10.parquet

─── Procesando: vibration/0Nm_BPFO_30.mat
✅ Convertido y subido a: vibration_temp_short/0Nm_BPFO_30.parquet

─── Procesando: vibration/0Nm_Misalign_01.mat
✅ Convertido y subido a: vibration_temp_short/0Nm_Misalign_01.parquet

─── Procesando: vibration/0Nm_Misalign_03.mat
✅ Convertido y subido a: vibration_temp_short/0Nm_Misalign_03.parquet

─── Procesando: vibration/0Nm_Misalign_05.mat
✅ Convertido y subido a: vibration_temp_short/0Nm_Misalign_05.par